# Client Summit: Using the Census API

## Interacting via Requests library vs. API client
Example: listing Platform tables in a given schema

#### HTTP request via Requests library

In [ ]:
import requests
import os

endpoint = 'https://api.civisanalytics.com/tables/'
api_key = os.environ.get('CIVIS_API_KEY')
parameters = {'schema': 'ts'}

response = requests.get(endpoint, 
                        auth=(api_key, ''),
                        params=parameters)

if response.status_code == 200:
    print(response.json())

#### via Civis Client

In [ ]:
import civis

client = civis.APIClient()

response = client.tables.list(schema='ts')
print(response)

## Using the Census API via the requests library

#### Documentation
- [requests](http://docs.python-requests.org/en/master/): For making API calls
- [civis](https://civis-python.readthedocs.io/en/latest/client.html): For interacting with the Civis API
- [us](https://pypi.python.org/pypi/us): For easily getting Census FIPS codes

In [ ]:
import requests
import civis
import us
import pandas as pd

# initialize Civis client
client = civis.APIClient()

In [ ]:
BASE_URL = 'https://api.census.gov/data/2016/acs/acs5'
# replace with your credential ID
CENSUS_API_KEY = client.credentials.get(9166).username

First, get data for every tract in DC.

Census Fields:
    - B08006_008E: number that commute via public transportation
    - B08006_001E: total number of employed people

In [ ]:
parameters = {'get': 'NAME,B08006_008E,B08006_001E', 
              'for': 'tract:*', 
              'in': 'state:' + us.states.DC.fips, 
              'key': CENSUS_API_KEY}

response = requests.get(BASE_URL, params=parameters)

In [ ]:
# view url the request is hitting
print(response.url)

In [ ]:
# view response status
print(response.status_code)

In [ ]:
# get response as json
data_dc = response.json()
data_dc

In [ ]:
# get column names from first row
cols = data_dc.pop(0)

# load to dataframe
df = pd.DataFrame(data_dc, columns=cols)
df

## Using the Census API via the datamade census API client
Documentation
- [census](https://github.com/datamade/census): Python wrapper for Census API

In [ ]:
from census import Census

# initialize census API client
c = Census(CENSUS_API_KEY)

In [ ]:
# get data for every tract in DC
fields = ('NAME', 'B08006_008E', 'B08006_001E')
data_dc = c.acs5.state_county_tract(fields, us.states.DC.fips, Census.ALL, Census.ALL)
data_dc

Note the different format of the data. Instead of a two-dimensional list, the `census` client returns a list of objects. 

This can be read more intuitively by `pandas`, so we don't have to pass in the list of column names when creating the dataframe:

In [ ]:
df = pd.DataFrame(data_dc)  
df

Now, let's get data for every tract nationally. 

`Census.ALL` gives you every geography of a certain type (county, tract, etc.). 

While the Census API allows you to get all sub-state geographies in a single call, it only lets you get one state at a time, so we'll have to add a loop.

In [ ]:
data_all = []
for state in us.STATES:
    data_all.extend(c.acs5.state_county_tract(fields, state.fips, Census.ALL, Census.ALL))

If the API you're using has a strict rate limit, you may want to include a delay of a second or two between calls:

In [ ]:
import time

data_all = []
for state in us.STATES:
    data_all.extend(c.acs5.state_county_tract(fields, state.fips, Census.ALL, Census.ALL))
    time.sleep(1)

In [ ]:
df = pd.DataFrame(data_all)  
df

## Loading the data to Civis Platform
This step will work the same regardless of how you get the data.

In [ ]:
import civis

civis.io.dataframe_to_civis(df, 'Civis Database', 'scratch.census_commute', 
                            existing_table_rows='drop').result()

Query the data!

In [ ]:
%%civisquery Civis Database
select * from scratch.census_commute limit 10